# Experimento:

Buscamos lograr contestar la pregunta: 
- ¿Es posible predecir la felicidad (positividad) de una canción en función de la popularidad (u otros parametros)?

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
dataframe = pd.read_excel('../Spotify.xlsx')
df_spotify = pd.DataFrame(dataframe)

#Ajustamos columnas para contraarrestar error de formato en archivo xlsx
df_spotify['duration_ms'] = df_spotify['duration_ms']/10
df_spotify['popularity'] = df_spotify['popularity']/10
df_spotify['streams'] = df_spotify['streams']/10
df_spotify['af_danceability'] = df_spotify['af_danceability']/1000
df_spotify['af_energy'] = df_spotify['af_energy']/1000
df_spotify['af_key'] = df_spotify['af_key']/10
df_spotify['af_loudness'] = df_spotify['af_loudness']/1000
df_spotify['af_speechiness'] = df_spotify['af_speechiness']/1000
df_spotify['af_acousticness'] = df_spotify['af_acousticness']/1000
df_spotify['af_instrumentalness'] = df_spotify['af_instrumentalness']/1000
df_spotify['af_liveness'] = df_spotify['af_liveness']/1000
df_spotify['af_valence'] = df_spotify['af_valence']/1000
df_spotify['af_tempo'] = df_spotify['af_tempo']/1000
df_spotify['af_time_signature'] = df_spotify['af_time_signature']/10

### Prediciendo con solo popularidad

En una primera instancia, experimentaremos sólo usando los atributos "streams" y "popularity" para intentar predecir "af_valence" (El cual representa la positividad o felicidad). Esto es porque de todos lo atributos, son estos dos los que se asocian con la "popularidad" de una canción dada.

In [4]:
df_util = df_spotify[["streams", "popularity", "af_valence"]]

Definimos una función que nos permitirá convertir los valores reales del atributo "af_valence" en etiquetas de texto mediante intervalos.

In [ ]:
def apply_etiqueta(elemento):
    if (elemento <= 0.25):
        return "Low"
    elif (elemento > 0.25) & (elemento <= 0.5):
        return "Medium-Low"
    elif (elemento > 0.5) & (elemento <= 0.75):
        return "Medium-High"
    else:
        return "High"

También 

In [ ]:
from sklearn.model_selection import train_test_split

df_etiquetado = df_util.copy()

df_etiquetado["af_valence"] = df_etiquetado["af_valence"].apply(apply_etiqueta)
X = df_etiquetado[["streams", "popularity"]]
y = df_etiquetado["af_valence"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)


## Dummy Classifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy = 'stratified')

dummy_clf.fit(X_train, y_train)

y_pred = dummy_clf.predict(X_val)

kn_acc = accuracy_score(y_val, y_pred)
print(classification_report(y_val, y_pred))


## K-Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

np.random.seed(42)

tuned_parameters = {'n_neighbors': list(range(1, 16, 1))}

score = 'f1_macro'

cls = KNeighborsClassifier()

clf = GridSearchCV(cls, param_grid = tuned_parameters, scoring = score, cv = 5)

clf.fit(X_train, y_train)

print("Mejor combinación de parámetros:")
print(clf.best_params_)

In [ ]:
kn_clf_1 = KNeighborsClassifier(n_neighbors=1)

kn_clf_1.fit(X_train, y_train)

y_pred = kn_clf_1.predict(X_val)

kn_acc = accuracy_score(y_val, y_pred)
print(classification_report(y_val, y_pred))

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

np.random.seed(42)

tuned_parameters = {'max_depth': list(range(1, 16, 1)), 'criterion': ['gini', 'entropy']}
score = 'f1_macro'
cls = DecisionTreeClassifier()

clf = GridSearchCV(cls, param_grid = tuned_parameters, scoring = score, cv = 5)

clf.fit(X_train, y_train)

print("Mejor combinación de parámetros:")
print(clf.best_params_)

In [ ]:
dtree_clf = DecisionTreeClassifier(max_depth = 15, criterion = 'gini')

dtree_clf.fit(X_train, y_train)

y_pred = dtree_clf.predict(X_val)

kn_acc = accuracy_score(y_val, y_pred)
print(classification_report(y_val, y_pred))

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb_clf = GaussianNB()

nb_clf.fit(X_train, y_train)

y_pred = nb_clf.predict(X_val)

kn_acc = accuracy_score(y_val, y_pred)
print(classification_report(y_val, y_pred))

## Support Vector Machines

In [ ]:
from sklearn.svm import SVC 

svm_clf = SVC()

svm_clf.fit(X_train, y_train)

y_pred = svm_clf.predict(X_val)

kn_acc = accuracy_score(y_val, y_pred)
print(classification_report(y_val, y_pred))